In [1]:
import sys
sys.path.append('../unseen')

import numpy as np
from dask.distributed import Client, LocalCluster
import xclim.analog

import myfuncs

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:46339 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 28.00 GiB


## Read data

In [4]:
cafe_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_awap-additive-correction_19911101-19921101_3650D_cafe-grid-TAS-POINT.zarr.zip'
ds_cafe = myfuncs.open_file(cafe_file)
ds_cafe          

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-11-01 00:00:00 1992-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) object dask.array<chunksize=(3650, 2), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(2, 3650, 96), meta=np.ndarray>
Attributes:
    history:  Wed Jun 23 13:32:39 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [5]:
awap_file = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'
ds_awap = myfuncs.open_file(awap_file, sel={'time': slice('1991-01-01', '1992-12-31')})
ds_awap

<xarray.Dataset>
Dimensions:  (time: 730)
Coordinates:
  * time     (time) object 1991-01-01 09:00:00 ... 1992-12-31 09:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(730,), meta=np.ndarray>
Attributes:
    history:  Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

## K-S test

In [6]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [7]:
fcst_stacked

<xarray.Dataset>
Dimensions:    (sample: 700800)
Coordinates:
    time       (sample) object dask.array<chunksize=(700800,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) object 1991-11-01 00:00:00 ... 1992-11-01 00:00:00
  - lead_time  (sample) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
Data variables:
    pr         (sample) float64 dask.array<chunksize=(700800,), meta=np.ndarray>
Attributes:
    history:  Wed Jun 23 13:32:39 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [8]:
fcst_stacked['sample'] = np.arange(fcst_stacked['sample'].size)

In [9]:
fcst_stacked

<xarray.Dataset>
Dimensions:  (sample: 700800)
Coordinates:
    time     (sample) object dask.array<chunksize=(700800,), meta=np.ndarray>
  * sample   (sample) int64 0 1 2 3 4 5 ... 700795 700796 700797 700798 700799
Data variables:
    pr       (sample) float64 dask.array<chunksize=(700800,), meta=np.ndarray>
Attributes:
    history:  Wed Jun 23 13:32:39 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [13]:
awap_faux_stacked = ds_awap.rename(time='sample')
awap_faux_stacked['sample'] = np.arange(awap_faux_stacked['sample'].size)
awap_faux_stacked

<xarray.Dataset>
Dimensions:  (sample: 730)
Coordinates:
  * sample   (sample) int64 0 1 2 3 4 5 6 7 ... 722 723 724 725 726 727 728 729
Data variables:
    pr       (sample) float64 dask.array<chunksize=(730,), meta=np.ndarray>
Attributes:
    history:  Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [22]:
test = xclim.analog.spatial_analogs(target=awap_faux_stacked,
                                    candidates=fcst_stacked.isel({'sample': slice(0, 730)}),
                                    dist_dim=['sample'],
                                    method='kolmogorov_smirnov')

In [19]:
test.values

array(0.3260274)

TODO: To look at the whole stacked forecast array we'd need to pad the observations with zeros so the sample dimension was the same size, or stack the obs by init_date and lead_time and then duplicate for the number of ensemble members...

In [20]:
xclim.analog.spatial_analogs(target=awap_faux_stacked,
                             candidates=fcst_stacked,
                             dist_dim=['sample'],
                             method='kolmogorov_smirnov')

ValueError: indexes along dimension 'dist' are not equal

## Temporal resampling

In [7]:
def resample_forecasts_monthly(ds, n_months, end_month, method='sum', 
                               lead_time_func='max'):
    """ Resample forecasts to a set of months for each year"""
    def _resample_grouped_monthly_forecasts(ds, method):
        month = np.unique(ds.init_date.dt.month)
        assert len(month) == 1
        start_month = (end_month - (n_months-1)) % 12
        i_start = (start_month - month[0]) % 12
        rs = ds.isel(lead_time=slice(i_start, None)).coarsen(
            lead_time=n_months, boundary='trim',
            coord_func={'lead_time': lead_time_func, 
                        'time': 'max'})
        if method == 'sum':
            return rs.sum()
        elif method == 'mean':
            return rs.mean()
        else:
            raise InputError('Unrecognised method')
    
    rs = ds.groupby('init_date.month').map(
        _resample_grouped_monthly_forecasts, method=method)
    rs['time'].attrs = ds['time'].attrs
    return rs

In [10]:
N_MONTHS = 3
END_MONTH = 11
ACC_METHOD = 'sum'
OBS_METHOD = 'Q-NOV'

fcst_acc = resample_forecasts_monthly(ds_cafe,
                                      n_months=N_MONTHS,
                                      end_month=END_MONTH,
                                      lead_time_func='max', #max_month_to_year,
                                      method=ACC_METHOD)

In [11]:
ds_cafe['lead_time']

<xarray.DataArray 'lead_time' (lead_time: 3650)>
array([   0,    1,    2, ..., 3647, 3648, 3649])
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
Attributes:
    units:    D

In [12]:
fcst_acc['lead_time']

<xarray.DataArray 'lead_time' (lead_time: 1213)>
array([  12,   15,   18, ..., 3642, 3645, 3648])
Coordinates:
  * lead_time  (lead_time) int64 12 15 18 21 24 27 ... 3636 3639 3642 3645 3648
Attributes:
    units:    D

In [13]:
fcst_acc['time'][0:10, 0].values

array([cftime.datetime(1991, 11, 13, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 11, 16, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 11, 19, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 11, 22, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 11, 25, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 11, 28, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 12, 1, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 12, 4, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 12, 7, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=False),
       cftime.datetime(1991, 12, 10, 0, 0, 0, 0, calendar='proleptic_gregori

In [14]:
obsv_acc = getattr(ds_awap.resample(time=OBS_METHOD), ACC_METHOD)()

In [15]:
obsv_acc['time']

<xarray.DataArray 'time' (time: 9)>
array([cftime.DatetimeProlepticGregorian(1991, 2, 28, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1991, 5, 31, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1991, 8, 31, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1991, 11, 30, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1992, 2, 29, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1992, 5, 31, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1992, 8, 31, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1992, 11, 30, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeProlepticGregorian(1993, 2, 28, 0, 0, 0, 0, has_year_zero=False)],
      dtype=object)
Coordinates:
  * time     (time) object 1991-02-28 00:00:00 ... 1993-02-28 00:00:00

In [32]:
fcst_acc = fcst_acc.where(fcst_acc.time.dt.month == END_MONTH, drop=True)
obsv_acc = obsv_acc.where(obsv_acc.time.dt.month == END_MONTH, drop=True)

In [33]:
fcst_acc

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 4, lead_time: 98)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) datetime64[ns] 1990-11-01 1991-11-01 ... 1993-11-01
  * lead_time  (lead_time) int64 12 15 18 21 24 27 ... 3303 3306 3309 3312 3315
    time       (lead_time, init_date) datetime64[ns] dask.array<chunksize=(98, 4), meta=np.ndarray>
Data variables:
    precip     (init_date, lead_time, ensemble) float64 dask.array<chunksize=(4, 98, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    history:    Thu Jun 10 15:27:22 2021: /g/data/e14/dbi599/miniconda3/envs/...
    title:      AccessOcean-AM2

In [34]:
obsv_acc

<xarray.Dataset>
Dimensions:  (time: 4)
Coordinates:
  * time     (time) object 1990-11-30 00:00:00 ... 1993-11-30 00:00:00
Data variables:
    precip   (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [7]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})
fcst_stacked

<xarray.Dataset>
Dimensions:    (sample: 1401600)
Coordinates:
    time       (sample) datetime64[ns] dask.array<chunksize=(1401600,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) datetime64[ns] 1990-11-01 1990-11-01 ... 1993-11-01
  - lead_time  (sample) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
Data variables:
    precip     (sample) float64 dask.array<chunksize=(1401600,), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    history:    Thu Jun 10 15:27:22 2021: /g/data/e14/dbi599/miniconda3/envs/...
    title:      AccessOcean-AM2

In [8]:
#ds_stacked = ds_stacked.assign_coords({'time': ds_stacked['time']})

In [18]:
obs_stacked = ds_awap.stack(sample=['time'])

In [19]:
obs_stacked

<xarray.Dataset>
Dimensions:  (sample: 1460)
Coordinates:
  * sample   (sample) MultiIndex
  - time     (sample) object 1990-01-01 09:00:00 ... 1993-12-31 09:00:00
Data variables:
    precip   (sample) float64 dask.array<chunksize=(1460,), meta=np.ndarray>

In [19]:
def unseen_univariate_moment_fidelity(fcst, obsv, period, alpha=5, n_repeats=1000, n_block=1, by_lead=True):
    """ Run unseen fidelity test
        if alpha == None, returns two Datasets
            n_repeats samples of the moments from the model
            moments from the observations
        else returns single Dataset with
            0 where obsv falls between alpha/2 % and 100-alpha/2 % of the bootstrapped model distribution
            -1 where the obsv falls above 100-alpha/2 %
            1 where the obsv falls below alpha/2 %
    """
        
    fcst_period_stacked = stack_super_ensemble(fcst, period, 
                                               super_ensemble_dims=('init_date','ensemble') 
                                               if by_lead else ('init_date','ensemble', 'lead_time'))
    obsv_period = obsv.where((obsv.time >= period[0]) & (obsv.time <= period[1]), drop=True)
    
    fcst_moments = n_random_resamples(fcst_period_stacked, 
                                      samples={'super_ensemble': (len(obsv_period['time']), n_block)}, 
                                      n_repeats=n_repeats,
                                      function=get_first_four_moments,
                                      function_kwargs={'dim':'super_ensemble'})

    obsv_moments = get_first_four_moments(obsv_period, 'time')
    
    if alpha:
        fcst_moments_upper = fcst_moments.quantile(1-alpha/200, dim='k')
        fcst_moments_lower = fcst_moments.quantile(alpha/200, dim='k')

        not_nan = (obsv_moments * fcst_moments_lower * fcst_moments_upper).notnull()
        above_min = obsv_moments >= fcst_moments_lower
        below_max = obsv_moments <= fcst_moments_upper

        passed = above_min & below_max
        too_low = (above_min == False) & (passed == False) 
        too_high = (below_max == False) & (passed == False)

        return (xr.where(passed, 2, 0) + xr.where(too_low, 3, 0) + xr.where(too_high, 1, 0) - 2).where(not_nan)
    else:
        return fcst_moments, obsv_moments

In [20]:
def get_first_four_moments(da, dim):
    """ Return the mean, std, skewness and kurtosis along dim """
    
    ds = da.mean(dim).to_dataset(name='mean')
    daf = da - da.mean(dim)
    ds['std'] = da.std(dim)
    ds['skew'] = ((daf ** 3).mean(dim) / ((daf ** 2).mean(dim) ** (3/2)))
    ds['kurt'] = ((daf ** 4).mean(dim) / ((daf ** 2).mean(dim) ** (2)))
    
    return ds

In [21]:
awap_moments = get_first_four_moments(da_awap, 'time')

In [29]:
awap_moments['mean'].data.shape

()

In [10]:
def random_resample(*objects, samples,
                    function=None, function_kwargs=None, replace=True):
    """
        Randomly resample from provided xarray objects and return the results of the subsampled dataset passed through \
        a provided function
                
        Parameters
        ----------
        *objects : xarray DataArray or Dataset
            Objects containing data to be resampled. The coordinates of the first object are used for resampling and the \
            same resampling is applied to all objects
        samples : dictionary
            Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
            within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}. The first \
            object in objects must contain all dimensions listed in samples, but subsequent objects need not.
        function : function object, optional
            Function to reduced the subsampled data
        function_kwargs : dictionary, optional
            Keyword arguments to provide to function
        replace : boolean, optional
            Whether the sample is with or without replacement
                
        Returns
        -------
        sample : xarray DataArray or Dataset
            Array containing the results of passing the subsampled data through function
    """
    objects_sub = [obj.copy() for obj in objects]
    for dimension, (n_samples, block_size) in samples.items():
        n_blocks = int(n_samples / block_size)
        n_samples = n_blocks * block_size
        random_samples = [slice(x,x+block_size) for x in np.random.choice(len(objects_sub[0][dimension])-block_size+1, 
                                                                          size=n_blocks,
                                                                          replace=replace)]
        objects_sub = [xr.concat([obj.isel({dimension: random_sample}) for random_sample in random_samples],
                                 dim=dimension) 
                       if dimension in obj.dims else obj 
                       for obj in objects_sub]
    
    if function:
        if function_kwargs:
            res = tuple([function(obj, **function_kwargs) for obj in objects_sub])
        else:
            res = tuple([function(obj) for obj in objects_sub])
    else:
        res = tuple(objects_sub)
    
    if len(res) == 1:
        return res[0]
    else:
        return res
    

def n_random_resamples(*objects, samples, n_repeats, 
                       function=None, function_kwargs=None,
                       replace=True, with_dask=True):
    """
    Repeatedly randomly resample from provided xarray objects and return
    the results of the subsampled dataset passed through a provided function
                
    Parameters
    ----------
    objects : xarray DataArray or Dataset
        Objects containing data to be resampled.
        The coordinates of the first object are used for resampling and
        the same resampling is applied to all objects
    samples : dictionary
        Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
        within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}
    n_repeats : int
        Number of times to repeat the resampling process
    function : function object, optional
        Function to reduced the subsampled data
    function_kwargs : dictionary, optional
        Keyword arguments to provide to function
    replace : boolean, optional
        Whether the sample is with or without replacement
    with_dask : boolean, optional
        If True, use dask to parallelize across n_repeats using dask.delayed
    write_to_disk
                
    Returns
    -------
    sample : xarray DataArray or Dataset
        Array containing the results of passing the subsampled data through function
    """

    if with_dask & (n_repeats > 1000):
        n_objects = itertools.repeat(objects[0], times=n_repeats)
        b = db.from_sequence(n_objects, npartitions=100)
        rs_list = b.map(random_resample, *(objects[1:]), 
                        **{'samples':samples, 'function':function, 
                           'function_kwargs':function_kwargs, 'replace':replace}).compute()
    else:              
        resample_ = dask.delayed(random_resample) if with_dask else random_resample
        rs_list = [resample_(*objects,
                             samples=samples,
                             function=function,
                             function_kwargs=function_kwargs,
                             replace=replace) for _ in range(n_repeats)] 
        if with_dask:
            rs_list = dask.compute(rs_list)[0]
    
    if len(objects) == 1:
        return xr.concat([r.unify_chunks() for r in rs_list], dim='k')
    else:
        return tuple([xr.concat([r.unify_chunks() for r in rs], dim='k') for rs in zip(*rs_list)])

In [ ]:
alpha = 5
n_repeats = 100
n_block = 1

fcst_moments = n_random_resamples(ds_stacked, 
                                  samples={'sample': (len(da_awap['time']), n_block)}, 
                                  n_repeats=n_repeats,
                                  function=get_first_four_moments,
                                  function_kwargs={'dim' : 'sample'})